## Creating a Relational Database From Spreadsheets

Moving from three CSVs to a small relational database.

Download the data here: https://figshare.com/articles/Portal_Project_Teaching_Database/1314459

DB Browser for SQLite here: https://sqlitebrowser.org/

Lesson Adapted from:
http://benbestphd.com/R-ecology-lesson/r-and-sql.html#introduction

#### Instructions for libraries

In [1]:
install.packages('RSQLite')

package 'RSQLite' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\jkenyon\AppData\Local\Temp\RtmpEnklld\downloaded_packages


In [2]:
library(RSQLite)

#### Set your working directory where the data is located, and where we will create the database

In [3]:
setwd("wlf550_data/March_03/")

#### Reading in data

In this case, we have three CSVs: species - with species information; surveys - with observations of species; and plots - information about specific study plots.

We will read each CSV/spreadsheet in as its own dataframe.

In [6]:
species <- read.csv("species.csv", stringsAsFactors=FALSE)
surveys <- read.csv("surveys.csv", stringsAsFactors=FALSE)
plots <- read.csv("plots.csv", stringsAsFactors=FALSE)

In [8]:
nrow(species)
head(species)
nrow(surveys)
head(surveys)
nrow(plots)
head(plots)

[1] 54

species_id,genus,species,taxa
AB,Amphispiza,bilineata,Bird
AH,Ammospermophilus,harrisi,Rodent
AS,Ammodramus,savannarum,Bird
BA,Baiomys,taylori,Rodent
CB,Campylorhynchus,brunneicapillus,Bird
CM,Calamospiza,melanocorys,Bird


[1] 35549

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
1,7,16,1977,2,NL,M,32,NA
2,7,16,1977,3,NL,M,33,NA
3,7,16,1977,2,DM,F,37,NA
4,7,16,1977,7,DM,M,36,NA
5,7,16,1977,3,DM,M,35,NA
6,7,16,1977,1,PF,M,14,NA


[1] 24

plot_id,plot_type
1,Spectab exclosure
2,Control
3,Long-term Krat Exclosure
4,Control
5,Rodent Exclosure
6,Short-term Krat Exclosure


## Creating a relational database

* We start by creating the file/filename.
* Then, we create a database connection, which accesses the file.
* The db connection is the primary object we call throughout our code.

In [9]:
myDB <- "wlf550.db"

In [38]:
myConn <- dbConnect(drv = SQLite(), dbname= myDB)

#### Writing a CSV to create a table

In [10]:
dbWriteTable(myConn,"species",species)

In [11]:
dbListTables(myConn)

[1] "species"

In [12]:
dbWriteTable(myConn,"surveys",surveys)

In [13]:
dbWriteTable(myConn,"plots",plots)

In [39]:
dbListTables(myConn)

[1] "plots"   "species" "surveys"

#### Querying the Database using SQL


SQL = Structured Query Language<br>
(Adapted from: https://www.dofactory.com/sql/)

* SELECT: The SELECT statement retrieves data from a database. The asterisk \* means return everything from the matching row.<br>
* FROM: The FROM term needs to be followed by the table-name.<br>
* LIMIT: The LIMIT term places a restriction on how many results are to be returned, specified by an integer.

In [15]:
# Query: retrieve all information from the first 10 rows of the species table

dbGetQuery(myConn, "SELECT * FROM species LIMIT 10")

species_id,genus,species,taxa
AB,Amphispiza,bilineata,Bird
AH,Ammospermophilus,harrisi,Rodent
AS,Ammodramus,savannarum,Bird
BA,Baiomys,taylori,Rodent
CB,Campylorhynchus,brunneicapillus,Bird
CM,Calamospiza,melanocorys,Bird
CQ,Callipepla,squamata,Bird
CS,Crotalus,scutalatus,Reptile
CT,Cnemidophorus,tigris,Reptile
CU,Cnemidophorus,uniparens,Reptile


In [16]:
tenSpeciesRows <- dbGetQuery(myConn, "SELECT * FROM species LIMIT 10")

In [17]:
typeof(tenSpeciesRows)
dim(tenSpeciesRows)

[1] "list"

[1] 10  4

In [18]:
dbListFields(myConn, "species")

[1] "species_id" "genus"      "species"    "taxa"

In [19]:
# Query: retrieve the record id number and the associated hindfoot length for the first 10 observations in the survey table.

dbGetQuery(myConn, "SELECT * FROM surveys LIMIT 10")

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
1,7,16,1977,2,NL,M,32,NA
2,7,16,1977,3,NL,M,33,NA
3,7,16,1977,2,DM,F,37,NA
4,7,16,1977,7,DM,M,36,NA
5,7,16,1977,3,DM,M,35,NA
6,7,16,1977,1,PF,M,14,NA
7,7,16,1977,2,PE,F,NA,NA
8,7,16,1977,1,DM,M,37,NA
9,7,16,1977,1,DM,F,34,NA
10,7,16,1977,6,PF,F,20,NA


### Conditional Queries

Select queries can operate against specific parameters.
* WHERE: The WHERE term must be followed by a column-name, as in retrieve data where a field  matches a particular pattern.
* CONDITIONS: Standard operators of =, >, <, etc. apply. Boolean operators of AND, OR, NOT also apply.

In [21]:
# Query: retrieve all records from the surveys table where hindfoot length is greater than 20.
# Query: retrieve all records from the species table where the genus starts with Crot.

#dbGetQuery(myConn, "SELECT * FROM surveys WHERE hindfoot_length > 20")
dbGetQuery(myConn, "SELECT * FROM species WHERE genus LIKE '%Crot%'")

species_id,genus,species,taxa
CS,Crotalus,scutalatus,Reptile
CV,Crotalus,viridis,Reptile


In [22]:
dbGetQuery(myConn,"SELECT count(*) FROM surveys")

count(*)
35549


In [23]:
dbGetQuery(myConn, "SELECT * FROM surveys WHERE species_id = 'PP' LIMIT 40")
#dbGetQuery(myConn, "SELECT * FROM species LIMIT 40")

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight
18,7,16,1977,2,PP,M,22,NA
29,7,17,1977,11,PP,M,NA,NA
49,7,18,1977,12,PP,F,17,NA
89,8,20,1977,12,PP,F,20,15
108,8,20,1977,11,PP,M,21,NA
163,9,11,1977,12,PP,F,21,16
217,9,13,1977,3,PP,F,19,15
897,5,18,1978,11,PP,M,21,15
933,5,19,1978,2,PP,F,20,13
981,6,8,1978,11,PP,M,21,15


### SQL Joins
More Resources: https://www.dofactory.com/sql/join

JOINs combine records from two tables. For example,  you might want to subset some data from table A and append to it associated data from table B.

Below are ways to think about the different types of JOINs. The most common is the INNER JOIN.

<img src="https://www.dofactory.com/Images/sql-joins.png" style="background-color:white">



#### Query:  Retrieve the genera where observed hindfoot length is less than 20.

In [26]:
query_species <- "SELECT genus FROM surveys WHERE hindfoot_length < 20"

In [27]:
dbGetQuery(myConn, query_species)

ERROR: Error in rsqlite_send_query(conn@ptr, statement): no such column: genus


In [30]:
query_species <- "
SELECT species.genus, count(*)
FROM species 
JOIN surveys
ON species.species_id = surveys.species_id
WHERE surveys.hindfoot_length < 20
GROUP BY species.genus"

In [31]:
dbGetQuery(myConn, query_species)

genus,count(*)
Baiomys,45
Chaetodipus,88
Dipodomys,1
Onychomys,509
Perognathus,1475
Peromyscus,399
Reithrodontomys,2515
Sigmodon,2


#### Query:  Retrieve the count of each genus for each plot type

In [32]:
plotType_query <- "SELECT d.plot_type, c.genus, count(*) AS count
FROM
(SELECT a.genus, b.plot_id
FROM species a
JOIN surveys b
ON a.species_id = b.species_id) c
JOIN plots d
ON c.plot_id = d.plot_id
GROUP BY d.plot_type, c.genus
ORDER BY d.plot_type, count DESC"

In [33]:
plotType_query <- "SELECT d.plot_type, c.genus, count(*) AS count
FROM
(SELECT a.genus, b.plot_id
FROM species a
JOIN surveys b
ON a.species_id = b.species_id) c
JOIN plots d
ON c.plot_id = d.plot_id
GROUP BY d.plot_type, c.genus
ORDER BY d.plot_type, count DESC"

In [ ]:
result <- dbGetQuery(myConn, plotType_query)
result

#### Query: For every other year in the dataset, return the number of observations of rodents

In [35]:
yearRange <- dbGetQuery(myConn, "SELECT min(year),max(year) FROM surveys")
years <- seq(yearRange[,1],yearRange[,2],by=2)

In [36]:
year_query <- paste("
SELECT a.year, b.taxa, count(*) as count
FROM surveys AS a
JOIN species AS b
ON a.species_id = b.species_id
AND b.taxa = 'Rodent'
AND a.year in (",
paste(years,collapse=",")
,")
GROUP BY a.year, b.taxa",
sep = "")

rCount <- dbGetQuery(myConn, year_query)
rCount

year,taxa,count
1977,Rodent,487
1979,Rodent,658
1981,Rodent,1394
1983,Rodent,1594
1985,Rodent,1398
1987,Rodent,1618
1989,Rodent,1522
1991,Rodent,1306
1993,Rodent,697
1995,Rodent,1168


In [145]:
write.csv(rCount,"2020-03-03_WLF550_programmaticQueryResult.csv", row.names = FALSE)

In [37]:
dbDisconnect(myConn)